# Welcome to the Synergy Roundtable
created by Stephan Koch & Dirk Derichsweiler - 01/11/2018<br>
do not hesitate to contact us: stephan.koch@hpe.com or dirk.derichsweiler@hpe.com



jupyter Notebook can be found: https://github.com/dderichswei/synergy


#### requirements
- Anaconda (Python) recommended (for the Jupyter installation and pip)
- Jupyter-Powershell (https://github.com/vors/jupyter-powershell)


#### additional information

On your HPE OneView appliance, or online<br>
https://10.0.20.50/help/cic-rest/en/content/index.html#home.html<br>
https://10.0.20.50/api-docs/current/ <br>
http://www.hpe.com/info/oneview/docs <br>
https://developer.hpe.com/

#### Powershell (POSH) specific
https://github.com/HewlettPackard/POSH-HPOneView<br>



### Installation of HP OneView Module
only required, if not installed yet.

In [1]:
Start-Process powershell -ArgumentList "Install-Module HPOneView.410 -AllowClobber -Verb runAs"
Start-Process powershell -ArgumentList "Set-ExecutionPolicy RemoteSigned -Scope CurrentUser"

Start-Process powershell -ArgumentList "Install-Module HPOneView.410 -AllowClobber -Verb runAs"
Start-Process powershell -ArgumentList "Set-ExecutionPolicy RemoteSigned -Scope CurrentUser"



### Login

<img src="Pictures/Login.gif" height="500" width="500" align="left">

#### import the PowerShell module:

In [2]:
Import-Module -name hponeview.410 

#### login:

In [4]:
Connect-HPOVMgmt -Hostname 10.0.20.50 -AuthLoginDomain local -UserName python -Password python123!


ConnectionID Name       UserName AuthLoginDomain Default
------------ ----       -------- --------------- -------
1            10.0.20.50 python   local           True   




#### ADVANCED: which functions are available?

In [ ]:
Get-Command -Module HPOneView.410 

### show existing networks

<img src="Pictures/show_network.gif" height="500" width="500" align="left">
<br>

In [ ]:
Get-HPOVNetwork

### create network

<img src="Pictures/create_network.gif" height="500" width="500" align="left">

In [6]:
New-HPOVNetwork -Name "Roundtable - Test Ethernet Network" -VlanId 200 -Type Ethernet -VLANType Tagged -Purpose General -SmartLink $False -PrivateNetwork $False  

New-HPOVNetwork : One or more networks failed the creation attempt!
At line:1 char:1
+ New-HPOVNetwork -Name "Roundtable - Test Ethernet Network" -VlanId 20 ...
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : NotSpecified: (:) [Write-Error], WriteErrorException
    + FullyQualifiedErrorId : Microsoft.PowerShell.Commands.WriteErrorException,New-HPOVNetwork
 

Name                               Status Details                                                                     
----                               ------ -------                                                                     
Roundtable - Test Ethernet Network Failed A network with the name Roundtable - Test Ethernet Network already exists.  




### create bulk network
How to setup multiple networks at once.

In [ ]:
$net = @(1,2,3,4,5,7,100,101,102,103,104,105,106,107,108,109,200)  
foreach ($i in $net) { New-HPOVNetwork -Name "Bulk_$i" -VlanId $i -Type Ethernet -VLANType Tagged } 

### delete bulk network
as it´s not needed for the demo

In [ ]:
Get-HPOVNetwork -name "bulk*" | Remove-HPOVNetwork -Confirm:$false

### show configured/existing storage (systems and pools)

<img src="Pictures/show_storage.gif" height="500" width="500" align="left">

In [ ]:
Get-HPOVStorageSystem


In [ ]:
Get-HPOVStoragePool  | select Name, URI 

### show volume templates

In [ ]:
Get-HPOVStorageVolumeTemplate

### create volume template

<img src="Pictures/create_volume_template.gif" height="500" width="500" align="left">

In [ ]:
$StorPool=Get-HPOVStoragePool -Name "FC_r5" -StorageSystem 3par20k 
New-HPOVStorageVolumeTemplate -Name "Roundtable Volume Template" -StoragePool $StorPool -Capacity 10240 

### show Enclosure / Server Hardware / Bay
<img src="Pictures/show_server_hw.gif" height="500" width="500" align="left">

In [ ]:
Get-HPOVEnclosureGroup 

In [ ]:
Get-HPOVServerHardwareType

In [8]:
Get-HPOVServer | select Name, URI


name                     uri                                                       
----                     ---                                                       
CTC Synergy HE01, bay 1  /rest/server-hardware/39313738-3134-5A43-4A37-343030373531
CTC Synergy HE01, bay 2  /rest/server-hardware/39313738-3134-5A43-4A37-343030373535
CTC Synergy HE01, bay 3  /rest/server-hardware/39313738-3134-5A43-4A37-343030373536
CTC Synergy HE01, bay 7  /rest/server-hardware/39313738-3134-5A43-4A37-343030373534
CTC Synergy HE01, bay 8  /rest/server-hardware/39313738-3134-5A43-4A37-343030373532
CTC Synergy HE01, bay 9  /rest/server-hardware/39313738-3134-5A43-4A37-343030373533
CTC Synergy HE11, bay 10 /rest/server-hardware/33323337-3235-5A43-3337-303948545656
CTC Synergy HE11, bay 2  /rest/server-hardware/33323337-3235-5A43-3337-30394854524D
CTC Synergy HE11, bay 3  /rest/server-hardware/33323337-3235-5A43-3337-303948545332
CTC Synergy HE11, bay 4  /rest/server-hardware/33323337-3235-5A43-3337-3039

#### show Serverprofile

In [ ]:
Get-HPOVServerProfileTemplate|select Name, URI 

In [ ]:
Get-HPOVServerProfile|select Name, URI

### create server profile (takes 2-3 minutes)
<br>
<img src="Pictures/boot_synergy.gif" height="500" width="500" align="left">

In [11]:
### Get the first available server based on the template configuration
##$Server = Get-HPOVServer -InputObject $ServerProfileTemplate -NoProfile | Select -First 1

$Server=Get-HPOVServer -Name "CTC Synergy HE11, bay 8"

#Power off Server if on
Stop-HPOVServer -Server $Server -Force -Confirm:$false | Wait-HPOVTaskComplete

#use * instead of "+" in string
$Template=Get-HPOVServerProfileTemplate -Name "CTC RHEL 7.6 AD*Docker SPT"
$OSDeploymentPlanName="CTC - RHEL 7.6 AD*Docker"

# Get the OS Deployment Plan
$OSDeploymentPlan = Get-HPOVOSDeploymentPlan -Name $OSDeploymentPlanName -ErrorAction Stop

# Get the associated deployment plan attributes
##$OSDeploymentAttributes = Get-HPOVOSDeploymentPlanAttribute -InputObject $OSDeploymentPlan -ErrorAction Stop
$OSDeploymentAttributes = Get-HPOVOSDeploymentPlanAttribute -InputObject $Template

($OSDeploymentAttributes | Where-Object name -eq "NewUser").value = "dirk"
($OSDeploymentAttributes | Where-Object name -eq "NewUserPassword").value = "HalloRoundTable!"

# Set root password, in clear text here, as OneView does not support secure string data types
#($OSDeploymentAttributes | Where-Object name -eq "NewRootPassword").value = [Runtime.InteropServices.Marshal]::PtrToStringAuto([Runtime.InteropServices.Marshal]::SecureStringToBSTR($Password))
($OSDeploymentAttributes | Where-Object name -eq "NewRootPassword").value = "Passw0rd"

# Set Hostname
#($OSDeploymentAttributes | Where-Object name -eq "Hostname").value = $ServerProfileName
($OSDeploymentAttributes | Where-Object name -eq "Hostname").value = "anyhostname"


$params = @{
        AssignmentType        = "Server";
        Description           = "HPE Synergy 480  Server ";
        Name                  = "Roundtable - API Demo Server (Stephan by POSH)";
        Server                = $Server;
        ServerProfileTemplate = $Template;
        OSDeploymentAttributes = $OSDeploymentAttributes;
        OSDeploymentPlan = $OSDeploymentPlan
}

New-HPOVServerProfile @params | Wait-HPOVTaskComplete

#Power on Server
Start-HPOVServer -Server $Server  



#Power off Server if on
Stop-HPOVServer -Server $Server -Force -Confirm:$false | Wait-HPOVTaskComplete

$Template=Get-HPOVServerProfileTemplate -Name "CTC RHEL 7.6 AD+Docker SPT"
Get-HPOVServerProfileTemplate : The specified Server Profile Template 'CTC RHEL 7.6 AD+Docker SPT' was not found on 
'10.0.20.50' appliance connection. Please check the name again, and try again.
At line:1 char:11
+ $Template=Get-HPOVServerProfileTemplate -Name "CTC RHEL 7.6 AD+Docker ...
+           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : ObjectNotFound: (Name:String) [Get-HPOVServerProfileTemplate], ServerProfileResourceExce 
   ption
    + FullyQualifiedErrorId : ServerProfileResourceNotFound,Get-HPOVServerProfileTemplate
 
$OSDeploymentPlanName="CTC - RHEL 7.6 AD+Docker"

# Get the OS Deployment Plan
$OSDeploymentPlan = Get-HPOVOSDeploymentPlan -Name $OSDeploymentPlanName -ErrorAction Stop
Get-HPOVOSDeploymentPlan : OS Deployment Plan "CTC - RHEL 7.6 AD+Do